In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
import json

In [ ]:
# Access Google Sheets
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('jsonFileFromGoogle.json', scope)

gc = gspread.authorize(credentials)

# URL key of the google sheet
spreadsheet_key = spreadsheet_key

In [ ]:
# Get the instance of the Spreadsheet
wkbook_title = 'Energy Consumption'
wkbook = gc.open(wkbook_title)

# Get the second sheet of the Spreadsheet
wksheet = wkbook.get_worksheet(1)

# Get all the records of the data
data = wksheet.get_all_records()

# View the data - in json format
# data

# Convert the json to dataframe
df = pd.DataFrame.from_dict(data)

# Check DF
df

,Date,Gas Reading (m3),Gas Reading (kWh),Gas Usage (kWh),Elec Reading (kWh),Elec Usage (kWh)
0,1-Jan-2021,5808,,,10121,
1,2-Jan-2021,,,,,
2,3-Jan-2021,,,,,
3,4-Jan-2021,,,,,
4,5-Jan-2021,,,,,
...,...,...,...,...,...,...
705,7-Dec-2022,,,,,
706,8-Dec-2022,,,,,
707,9-Dec-2022,,,,,
708,10-Dec-2022,,,,,


In [ ]:
# Check data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Date                710 non-null    object
 1   Gas Reading (m3)    710 non-null    object
 2   Gas Reading (kWh)   710 non-null    object
 3   Gas Usage (kWh)     710 non-null    object
 4   Elec Reading (kWh)  710 non-null    object
 5   Elec Usage (kWh)    710 non-null    object
dtypes: object(6)
memory usage: 33.4+ KB


In [ ]:
# Convert obj columns to num
cols = df.columns.drop('Date')
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

# Convert date obj column to datetime type
df['Date'] = pd.to_datetime(df['Date']).dt.date # returns object

In [ ]:
# Fill in missing values for gas reading
df["Gas Reading (m3)"].interpolate(method ='linear', limit_direction ='forward', inplace=True)
df["Gas Reading (m3)"] = df["Gas Reading (m3)"].astype(int)

In [ ]:
# Convert to kwh
df["Gas Reading (kWh)"] = df["Gas Reading (m3)"] * 10.5335
df["Gas Reading (kWh)"] = df["Gas Reading (kWh)"].astype(int)

,Date,Gas Reading (m3),Gas Reading (kWh),Gas Usage (kWh),Elec Reading (kWh),Elec Usage (kWh)
0,2021-01-01,5808,61178,NaN,10121.0,NaN
1,2021-01-02,5813,61231,NaN,NaN,NaN
2,2021-01-03,5818,61283,NaN,NaN,NaN
3,2021-01-04,5823,61336,NaN,NaN,NaN
4,2021-01-05,5829,61399,NaN,NaN,NaN
...,...,...,...,...,...,...
683,2022-11-15,7369,77621,NaN,NaN,NaN
684,2022-11-16,7369,77621,NaN,NaN,NaN
685,2022-11-17,7370,77631,NaN,NaN,NaN
686,2022-11-18,7371,77642,NaN,NaN,NaN


In [ ]:
# Calculate daily gas usage
next_day_reading = list(df["Gas Reading (kWh)"][1:])
next_day_reading.append(np.nan) # add a NaN to the end of the list to supress error

df["Gas Reading next day"] = next_day_reading # create helper column
df["Gas Reading next day"].interpolate(method ='linear', limit_direction ='forward', inplace=True) # fill in NaN at the end
df["Gas Usage (kWh)"] = df["Gas Reading next day"] - df["Gas Reading (kWh)"]

In [ ]:
# Fill missing values for electric reading
df["Elec Reading (kWh)"].interpolate(method ='linear', limit_direction ='forward', inplace=True)
df["Elec Reading (kWh)"] = df["Elec Reading (kWh)"].astype(int)

In [ ]:
# Calculate daily electric usage
next_day_reading = list(df["Elec Reading (kWh)"][1:])
next_day_reading.append(np.nan) # add a NaN to the end of the list to supress error

df["Elec Reading next day"] = next_day_reading
df["Elec Reading next day"].interpolate(method ='linear', limit_direction ='forward', inplace=True) # fill in NaN at the end
df["Elec Usage (kWh)"] = df["Elec Reading next day"] - df["Elec Reading (kWh)"]

In [ ]:
# Remove unnecessary columns
df = df.iloc[:, :-2]

In [ ]:
# Remove rows where date is after yesterday
from datetime import date
 
# Get today's date
today = date.today()

df.drop(df[df["Date"] >= today].index, inplace=True)

In [ ]:
# Check DF
df

,Date,Gas Reading (m3),Gas Reading (kWh),Gas Usage (kWh),Elec Reading (kWh),Elec Usage (kWh)
0,2021-01-01,5808,61178,53.0,10121,6.0
1,2021-01-02,5813,61231,52.0,10127,6.0
2,2021-01-03,5818,61283,53.0,10133,7.0
3,2021-01-04,5823,61336,63.0,10140,6.0
4,2021-01-05,5829,61399,53.0,10146,7.0
...,...,...,...,...,...,...
683,2022-11-15,7369,77621,0.0,13660,9.0
684,2022-11-16,7369,77621,10.0,13669,8.0
685,2022-11-17,7370,77631,11.0,13677,8.0
686,2022-11-18,7371,77642,10.0,13685,9.0


In [ ]:
# Send DF to Google Sheets
set_with_dataframe(wkbook.get_worksheet(2), df)